In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import pathlib
from IPython import get_ipython
import librosa

import ipynb.fs.defs.build_spectogram_ds as wave_to_spec



global history
global model
global test_ds
global label_names
global TEST_DIR


Let's plot the training and validation loss curves to check how your model has improved during training:


In [ ]:
def model_train_results():
    metrics = history.history
    plt.figure(figsize=(16,6))
    plt.subplot(1,2,1)
    print(metrics)
    plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
    # plt.plot(history.epoch, metrics['loss'])
    plt.legend(['loss', 'val_loss'])
    plt.ylim([0, max(plt.ylim())])
    plt.xlabel('Epoch')
    plt.ylabel('Loss [CrossEntropy]')

    plt.subplot(1,2,2)
    plt.plot(history.epoch, 100*np.array(metrics['accuracy']), 100*np.array(metrics['val_accuracy']))
    # plt.plot(history.epoch, 100*np.array(metrics['accuracy']))
    plt.legend(['accuracy', 'val_accuracy'])
    plt.ylim([0, 100])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy [%]')

In [ ]:
def confusion_matrix():
    y_pred = model.predict(test_ds)
    y_pred = tf.argmax(y_pred, axis=1)
    y_true = tf.concat(list(test_ds.map(lambda s,lab: lab)), axis=0)

    confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_mtx,
                xticklabels=label_names,
                yticklabels=label_names,
                annot=True, fmt='g')
    plt.xlabel('Prediction')
    plt.ylabel('Label')
    plt.show()

## Run inference on an audio file

Finally, verify the model's prediction output using an input audio file of someone saying "no". How well does your model perform?


In [ ]:
def visualize_audio(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=None)
    except Exception as e:
        print(f"Could not process file '{file_path}': {e}")
        return
    
    # Überprüfen, ob das Audio mehrdimensional ist (z.B. Stereo)
    if audio.ndim > 1:
        waveform = tf.squeeze(audio)
    else:
        waveform = audio 

    # Berechne die Anzahl der Samples, die eine Sekunde darstellen
    one_second_samples = sample_rate

    # Finde die Mitte des Audios und schneide eine Sekunde heraus
    mid_point = len(audio) // 2
    start = max(0, mid_point - one_second_samples // 2)
    end = start + one_second_samples
    audio_segment = audio[start:end]

    # Überprüfen, ob das Audio mehrdimensional ist (z.B. Stereo)
    if audio_segment.ndim > 1:
        waveform = tf.squeeze(audio_segment)
    else:
        waveform = audio_segment

    print(f"Form des Audiosignals: {waveform.shape}")
    print(f"Sample Rate: {sample_rate}")

    spectrogram = wave_to_spec.get_spectrogram(waveform)
    # spectrogram = wave_to_spec.get_mel_spectrogram(waveform)

    # Dimension anpassen für das Modell
    input_tensor = spectrogram[tf.newaxis, ...]

    prediction = model(input_tensor)

    x_labels = label_names  

    plt.bar(x_labels, tf.nn.softmax(prediction[0]))
    plt.title(f'Vorhersage für {os.path.basename(file_path)}')
    plt.show()

    display.display(display.Audio(waveform, rate=sample_rate))

def process_directory_for_visualization(directory_path):
    # wav_files = glob.glob(os.path.join(directory_path, "*.wav"))
    for root, dirs, files in os.walk(directory_path):
        for file in files:    
            if file.endswith(".wav"):
                file_path = os.path.join(root, file)
                print(f"Verarbeite Datei: {file_path}")
                visualize_audio(file_path)
            else:
                print("Keine WAV-Dateien im Verzeichnis gefunden.")
                return

# extract_zip(TEST_DIR, DATA_DIR)
# rename_audio_files(DATA_DIR)

In [ ]:
def run(_test_dir, _test_ds,_history, _model, _label_names):
    global TEST_DIR, history, model, test_ds, label_names
    
    TEST_DIR =_test_dir
    history =_history
    model = _model
    test_ds = _test_ds
    label_names = _label_names
    
    model_train_results()
    
    #Run the model on the test set and check the model's performance
    model.evaluate(test_ds, return_dict=True)
    
    confusion_matrix()
    process_directory_for_visualization(TEST_DIR)
    
    
    
    